In [11]:
!pip install js2py
!pip install geopandas

     |████████████████████████████████| 1.0 MB 6.6 MB/s 
     |████████████████████████████████| 6.3 MB 43.5 MB/s 
     |████████████████████████████████| 16.7 MB 319 kB/s 


In [12]:
import warnings
warnings.filterwarnings("ignore")

from IPython.display import display, HTML, IFrame
import math
import ast
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import MultiPoint
from shapely import wkt
import js2py
from scipy import stats
import shapely
import matplotlib.pyplot as plt
import bs4, re
import copy
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from urllib.request import urlopen
import requests
import json
%matplotlib inline

from pandas.io.json import json_normalize
import requests

In [13]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [14]:
def getBounds(txt):
  dictionary = {"{":'(', "}":')', " ":'', "[":'', "]":''}
  transTable = txt.maketrans(dictionary)
  txt = txt.translate(transTable)
  txt = txt.replace("'latitude':",'')
  txt = txt.replace("'longitude':",'')

  lst = list(ast.literal_eval(txt))
  points = MultiPoint(lst)
  b = points.bounds

  return b

In [15]:
def getPoly(txt):
  dictionary = {"{":'(', "}":')', " ":'', "[":'', "]":''}
  transTable = txt.maketrans(dictionary)
  txt = txt.translate(transTable)
  txt = txt.replace("'latitude':",'')
  txt = txt.replace("'longitude':",'')
  txt = txt.replace('"','')
  txt = txt.replace(",",' ')
  txt = txt.replace(") (",',')  

  return txt

In [16]:
import pickle
import pandas as pd

In [17]:
city = 'SF'
data_type = 'Training'

In [19]:
BusStats(city,data_type)

,Rank#,Name,Walk Score,Transit Score,Population,Count,geometry,area,perSqKm,perThouCap
0,1,Chinatown,100,99,8646,21,"POLYGON ((-1943329.640 -536875.296, -1943336.6...",3.182238e-01,65.99129,2.428869
1,2,Tenderloin,100,100,21848,54,"POLYGON ((-1944015.125 -538264.744, -1944392.4...",5.328562e-01,101.340654,2.471622
2,3,Polk Gulch,100,85,5096,28,"POLYGON ((-1944598.982 -536616.235, -1944666.3...",3.008141e-01,93.080737,5.494505
3,4,Downtown-Union Square,99,100,2112,33,"POLYGON ((-1943284.039 -537836.907, -1944015.1...",3.578189e-01,92.225436,15.625
4,5,North Beach,99,86,10684,81,"POLYGON ((-1943499.395 -535768.672, -1943509.1...",5.814414e-01,139.308967,7.58143
...,...,...,...,...,...,...,...,...,...,...
105,106,Midtown Terrace,49,69,1699,26,"POLYGON ((-1948048.682 -540534.478, -1948025.0...",5.466968e-01,47.55835,15.303119
106,107,Lakeshore,43,50,3541,149,"POLYGON ((-1952117.247 -541712.483, -1952182.5...",7.158641e+00,20.814005,42.078509
107,108,Candlestick Point State Recreation Area,41,64,1230,16,"POLYGON ((-1943636.132 -545695.024, -1943628.4...",1.611062e+00,9.931337,13.00813
108,109,Presidio National Park,40,54,3276,0,"POLYGON ((-1948378.721 -534038.970, -1948389.8...",1.792828e+07,0.0,0.0


In [18]:
def BusStats(city,data_type):
  #print('Running Bus Stop Density Calculations for ' + city + ' in ' + data_type)
  #import dataframe
  df_location = '/content/gdrive/MyDrive/Columbia/2 Spring 22/ML for Urban Data/ML Final/Project Files/Urban-Mobility-Index/Data/'+ data_type + '/' + city + '/' + city +'.pkl'
  df = pd.read_pickle(df_location)
  df["WKT"] = ""
  df["Count"] = ""
  df["WKTPoly"] = ""  

  i = 0
  for i in range(df.shape[0]):
    df["WKTPoly"][i] = copy.deepcopy(df["Polygon"][i])
    i = i + 1

  #reverse polygon
  i = 0

  for i in range(df.shape[0]):
    for j in range(len(df['Polygon'][i])):
      df['WKTPoly'][i][j] = str(dict(reversed(list(df['WKTPoly'][i][j].items()))))
    if df['WKTPoly'][i][0] != df['WKTPoly'][i][-1]:
      df['WKTPoly'][i].append(df['WKTPoly'][i][0])
    else:
      j = j + 1
  i = i + 1

  #format WKT column as per geopandas
  i = 0
  url = "https://z.overpass-api.de/api/interpreter"

  for i in range(df.shape[0]):
    p = df['WKTPoly'][i]
    df['WKT'][i] = 'POLYGON (' + getPoly(str(p)) + ')'
    #get bus stop nodes for polygon
    query = f"""
      [out:json][timeout:25];
      node["public_transport"="platform"]["bus"="yes"]
      {getBounds(str(df['Polygon'][i]))};
      out skel;
    """
    r = requests.get(url, params={'data': query})
    #handle errors
    try:
      json_normalize(r.json()['elements'])[['id']]
    except:
      df['Count'][i] = 0
    else:
      #Points Geopandas GeoDataFrame   
      bus = json_normalize(r.json()['elements'])[['id']]  # create a DataFrame from the data
      df['Count'][i] = int(bus.shape[0])

  #drop columns we don't need anymore
  df['geometry'] = df.WKT.apply(wkt.loads)
  df.drop(['Bike Score', 'URL','WKT','Polygon' ,'WKTPoly','EnPath'], axis=1, inplace=True) 

  # Polygon Geopandas GeoDataFrame
  eqArea_gdf = gpd.GeoDataFrame(df, geometry='geometry', crs = {'init': 'epsg:4326'}) #wgs84 projection - terrible for area calculations
  eqArea_gdf= eqArea_gdf.to_crs({'init': 'epsg:2163'}) #equal area projection 
  eqArea_gdf["area"] = eqArea_gdf['geometry'].area/ 10**6 #area in sq. km!

  # Calculate Fields
  eqArea_gdf["perSqKm"] = eqArea_gdf["Count"]/eqArea_gdf["area"] #per square kilometer
  eqArea_gdf['Population'] = eqArea_gdf['Population'].str.replace(',', '')
  eqArea_gdf['Population'] = eqArea_gdf['Population'].astype(int)
  eqArea_gdf["perThouCap"] = eqArea_gdf["Count"]*1000/eqArea_gdf["Population"] #per thousand capita

  return eqArea_gdf

  #Export file
  #pkl_out = '/content/gdrive/MyDrive/Columbia/2 Spring 22/ML for Urban Data/ML Final/Project Files/Urban-Mobility-Index/Data/' + data_type +'/'+ city + '/Bus Stops/Density.pkl'
  #eqArea_gdf.to_pickle(pkl_out)
  #print("Successfully Exported Bus Stop Density of " + city + ' to ' + data_type + '.')

In [ ]:
input_data = {'city':['Ann Arbor', 'Boulder', 'Chicago', 'DC', 'NYC', 'SF','Madison','Tulsa','Seattle'],
        'data_type':['Training','Training','Training','Training','Training','Training','Testing','Testing','Testing']}
input_df = pd.DataFrame(input_data)
input_df

,city,data_type
0,Ann Arbor,Training
1,Boulder,Training
2,Chicago,Training
3,DC,Training
4,NYC,Training
5,SF,Training
6,Madison,Testing
7,Tulsa,Testing
8,Seattle,Testing


In [ ]:
#n = 0

#for n in range(input_df.shape[0]):
#  city = input_df['city'][n]
#  data_type = input_df['data_type'][n]
#  BusStats(city,data_type)
#  n = n + 1